In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
img_size = 380
batch_size = 32

In [ ]:
basePath = '/kaggle/input/cassava-leaf-disease-classification/'
trainImagesPath = basePath + 'train_images/'
testImagesPath = basePath + 'test_images/'
train_info = pd.read_csv(basePath + 'train.csv')

In [ ]:
trainingSize = train_info.shape
print('The number of training examples are: ',trainingSize[0])

In [ ]:
import seaborn as sns
sns.countplot(train_info['label'])

It is obvious that the data is not distributed uniformly in the training set with more than half of the training images in class label 3. But the metric for the model is set to be categorization accuracy. Will it make sense?

In [ ]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
import tensorflow as tf
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
mixed_precision.set_policy(policy)

def buildModel(IMG_SIZE,NUM_CLASSES):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB4(include_top=False, input_tensor=inputs, weights="imagenet")
    #model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax",dtype="float32",name="pred")(x)
    
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
import matplotlib.pyplot as plt

def plot_hist(hist):
    plt.figure()
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

val_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
from sklearn.model_selection import train_test_split
train_info['label'] = train_info['label'].astype('str')
train_df, val_df = train_test_split(train_info, test_size=0.1, random_state=42)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,directory=trainImagesPath,
    x_col="image_id",y_col="label",
    target_size=(img_size, img_size), 
    batch_size=batch_size,class_mode = "sparse"
    ) 
val_generator = val_datagen.flow_from_dataframe(
    val_df,directory=trainImagesPath,
    x_col="image_id",y_col="label",
    target_size=(img_size, img_size),
    batch_size=batch_size,class_mode = "sparse"
    )     
model = buildModel(img_size,5)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.00001)
mcp_save = ModelCheckpoint('best_mdl.h5', save_best_only=True, monitor='val_loss', mode='min')
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=train_df.shape[0]  // batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_df.shape[0]  // batch_size,
    callbacks=[earlyStopping, mcp_save,reduceLROnPlat])
plot_hist(hist)